In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
orders = pd.read_csv("../input/open-shopee-code-league-logistic/delivery_orders_march.csv")
orders

In [ ]:
sla_matrix = pd.read_excel('../input/open-shopee-code-league-logistic/SLA_matrix.xlsx')
sla_matrix

In [ ]:
orders.dtypes

In [ ]:
sla_matrix.dtypes

In [ ]:
# orders['2nd_deliver_attempt'] = orders['2nd_deliver_attempt'].fillna(pd.to_datetime('1997-12-13'))
orders['2nd_deliver_attempt'] = orders['2nd_deliver_attempt'].fillna(0)

In [ ]:
holidays = ['2020-03-08', '2020-03-25', '2020-03-30', '2020-03-31']

In [ ]:
orders['pick'] = (pd.to_datetime(orders['pick'], unit = 's')
                                .dt.tz_localize('utc')
                                .dt.tz_convert('Asia/Hong_Kong'))
orders['1st_deliver_attempt'] = (pd.to_datetime(orders['1st_deliver_attempt'], unit = 's')
                                .dt.tz_localize('utc')
                                .dt.tz_convert('Asia/Hong_Kong'))
orders['2nd_deliver_attempt'] = (pd.to_datetime(orders['2nd_deliver_attempt'], unit = 's')
                                .dt.tz_localize('utc')
                                .dt.tz_convert('Asia/Hong_Kong'))

In [ ]:
orders['offset'] = 2
orders['offset'] = 3 if 'manila' in str(orders['buyeraddress']).lower() and orders['offset'] == 2 else orders['offset']
orders['offset'] = 5 if 'luzon' in str(orders['buyeraddress']).lower() and orders['offset'] == 2 else orders['offset']
orders['offset'] = 7 if 'visayas' in str(orders['buyeraddress']).lower() and orders['offset'] == 2 else orders['offset']
orders['offset'] = 7 if 'mindanao' in str(orders['buyeraddress']).lower() and orders['offset'] == 2 else orders['offset']

In [ ]:
# orders['sla'] = orders['pick'] + pd.to_timedelta(np.ceil(orders['offset']), unit="D")
orders['sla'] = pd.DatetimeIndex(orders['pick']) + pd.DateOffset(7)
# for i in range(0, len(orders['orderid'])):
#     if 'manila' in str(orders['buyeraddress'][i]).lower():
#         orders['sla'][i] = pd.DatetimeIndex(orders['pick'][i]) + pd.DateOffset(3)
#     elif 'luzon' in str(orders['buyeraddress'][i]).lower():
#         orders['sla'][i] = pd.DatetimeIndex(orders['pick'][i]) + pd.DateOffset(5)
#     elif 'visayas' in str(orders['buyeraddress'][i]).lower():
#         orders['sla'][i] = pd.DatetimeIndex(orders['pick'][i]) + pd.DateOffset(7)
#     else:
#         orders['sla'][i] = pd.DatetimeIndex(orders['pick'][i]) + pd.DateOffset(7)

In [ ]:
# orders['is_late'] = []
# orders['is_late'] = orders['1st_deliver_attempt'] - orders['pick'] <= 
# orders['is_late'] = (orders['1st_deliver_attempt'] > orders['sla']).astype(int)
orders['is_late'] = ((orders['1st_deliver_attempt'] > orders['sla']) | (orders['2nd_deliver_attempt'] > orders['sla'])).astype(int)

In [ ]:
header = ['orderid', 'is_late']

In [ ]:
submission = orders.to_csv('submission.csv', columns = header, index = False)

In [ ]:
submission_df = pd.read_csv('../working/submission.csv')
submission_df.shape
# submission_df.head()